In [12]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import matplotlib.pyplot as plt

## Dataset
Using `make_moons` we create a dataset containing 1000 samples, 800 dedicated to the train set and 200 to the test set. They are converted to tensors so they could be used via torch.

In [21]:
x, y = make_moons(n_samples=1000, noise=0.1, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

x_train, x_test, y_train, y_test

(tensor([[ 2.0433,  0.2758],
         [-1.0620,  0.4074],
         [-0.1452,  1.0842],
         ...,
         [-0.7968,  0.5879],
         [ 1.0717,  0.1348],
         [ 1.9319,  0.1316]]),
 tensor([[ 5.6148e-01, -4.0993e-01],
         [ 9.0668e-01,  5.9674e-01],
         [ 1.2998e+00, -4.2056e-01],
         [ 7.3457e-01,  8.9540e-01],
         [ 1.7914e+00, -2.0475e-01],
         [ 1.9957e+00, -3.7357e-02],
         [ 1.0631e+00,  2.3987e-01],
         [ 1.0658e+00,  1.0397e-01],
         [ 1.2102e+00, -7.4582e-01],
         [ 6.9790e-01,  6.0517e-01],
         [ 6.9864e-01,  6.8789e-01],
         [ 5.8863e-01, -4.1815e-01],
         [ 1.0638e+00,  3.1609e-01],
         [ 1.4488e+00, -6.5360e-01],
         [ 9.4124e-03,  1.1181e+00],
         [ 1.8996e+00,  3.7093e-01],
         [ 2.0101e-01, -1.1247e-01],
         [ 4.4677e-01,  9.3237e-01],
         [ 6.8076e-01, -4.4544e-01],
         [ 4.1857e-01,  9.9449e-01],
         [ 7.8490e-01,  6.6987e-01],
         [-9.1488e-01, -6.7186e-0

## RBF Network
There we have a network with `input_dim` inputs and `hidden_dim` hidden neurons and one output.

In [15]:
class RBF(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(RBF, self).__init__()
        self.centers = nn.Parameter(torch.randn(hidden_dim, input_dim))
        self.sigma = nn.Parameter(torch.ones(hidden_dim))
        self.linear = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        return self.linear(torch.exp(-(torch.cdist(x, self.centers) ** 2) / (self.sigma ** 2)))


## Train RBF
We Train the RBF Network considering given `hidden_dim`, `learning rate` and `epochs`.

In [22]:
def train(hidden_dim, lr, epochs):
    model = RBF(input_dim=2, hidden_dim=hidden_dim)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    losses = []
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    model.eval()
    y_test_pred = model(x_test).detach().numpy()
    mse = mean_squared_error(y_test, y_test_pred)

    y_test_binary = (y_test_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_test.numpy(), y_test_binary)
    
    return model, mse, losses, accuracy

## Let's Run the Model!

In [26]:
hidden_dims = [1, 2, 3, 4, 5, 10, 20, 40, 80, 100]
results = {}
lr = 0.01
epochs = 1000

for hidden_dim in hidden_dims:
    model, mse, losses, accuracy = train(hidden_dim=hidden_dim, lr=lr, epochs=epochs)
    results[hidden_dim] = (mse, losses, accuracy)
    print(f"Hidden Dimension: {hidden_dim}, MSE: {mse:.4f}, Accuracy: {accuracy:.4f}")

Hidden Dimension: 1, MSE: 0.0985, Accuracy: 0.8400
Hidden Dimension: 2, MSE: 0.0448, Accuracy: 0.9700
Hidden Dimension: 3, MSE: 0.0168, Accuracy: 1.0000
Hidden Dimension: 4, MSE: 0.0099, Accuracy: 1.0000
Hidden Dimension: 5, MSE: 0.0049, Accuracy: 1.0000
Hidden Dimension: 10, MSE: 0.0035, Accuracy: 1.0000
Hidden Dimension: 20, MSE: 0.0015, Accuracy: 1.0000
Hidden Dimension: 40, MSE: 0.0011, Accuracy: 1.0000
Hidden Dimension: 80, MSE: 0.0003, Accuracy: 1.0000
Hidden Dimension: 100, MSE: 0.0005, Accuracy: 1.0000
